In [ ]:
import pandas as pd
from sklearn.metrics import classification_report

In [ ]:
def error_fixer(df):
    pred = []
    gold = []
    errors = []
    for idx, row in df.iterrows():
        if len(row.semtag) < len(row.gen_semtag):
            pred += [i for i in row.gen_semtag[:len(row.semtag)]]
            gold += [i for i in row.semtag]
            errors.append([idx, 'longer', row.semtag, row.gen_semtag])
        elif len(row.semtag) > len(row.gen_semtag):
            errors.append([idx, 'shorter', row.semtag, row.gen_semtag])
        else:
            pred += [i for i in row.gen_semtag]
            gold += [i for i in row.semtag]
            
    return gold, pred

In [ ]:
def classification_printer(in_file):
    df = pd.read_json(in_file)

    gold, pred = error_fixer(df)

    print(classification_report(gold, pred))

In [ ]:
def mt5_other_lang_classification(in_file):

    df = pd.read_csv(in_file)

    grouped_sentences = df.groupby('sent_file').agg({'token': list,
                                                     'lemma': list,
                                                     'from': list,
                                                     'to': list,
                                                     'semtag': list}).reset_index()

    in_token = []
    out_semtag = []
    for index, data in grouped_sentences.iterrows():
        cur_token = []
        cur_semtag = []
        cur_extra_id = -1
        for word in data[1]:
            cur_extra_id += 1
            cur_token.append('<extra_id_{}> {} '.format(cur_extra_id, word))
            cur_semtag.append('<extra_id_{}> {} '.format(cur_extra_id,
                                                        data[-1][data[1].index(word)]))

        in_token.append(cur_token)
        out_semtag.append(cur_semtag)

    grouped_sentences['in_token'] = in_token
    grouped_sentences['out_semtag'] = out_semtag

    return grouped_sentences

In [ ]:
classification_printer('./')